## 영상 이진화

영상의 각 픽셀을 두 개의 분류로 나누는 작업. **영상의 픽셀 값을 0또는 255로 변환**하는 연산. 

영상에서 관심 있는 객체 영역과 배경 영역을 구분하기 위한 용도로 사용한다.

<br/>

![image](https://user-images.githubusercontent.com/44194558/140493441-8b4c80b2-d1ba-4d25-8ca3-c042ffa016d7.png)

   * 문서 스캔 영상에서 배경과 글자 영역을 구분
   
  
   * 지문 인식을 위한 전처리



기본적으로 영상의 각 픽셀 값을 이용. **그레이스케일 영상에 대해 영상의 픽셀값이 특정 값 보다 크면 255로, 작으면 0으로 설정**. 

임계값은 영상의 이진화를 수행하는 목적에 따라 적절하게 결정해야 한다. 

<br/>

![image](https://user-images.githubusercontent.com/44194558/140493795-546d8756-59a7-4d94-809b-16b960f097af.png)

  * 혈액 속 세포를 촬영한 영상. 세포 중에는 특정 약품에 의해 염색된 세포가 존재.
  
  
  * 영상의 히스토그램에는 두 개의 큰 분포가 형성되어 있는 것을 확인. (검은 색으로 염색된 픽셀의 분포가 존재)
  
  
  * 임계값 T1, T2에 따라 서로 다른 의미를 갖는 이진화 영상을 얻을 수 있음.


**임계값 함수**

cv2.threshold(src, thresh, maxval, type, dst=None) -> retval, dst

`src` : 입력 영상.

`thresh` : 사용자 지정 임계값

`maxval` : 보통 255로 지정.

`type` : 임계값 함수 동작 지정 또는 자동 임계값 결정 방법 지정

`retval` : 사용된 임계값

`dst` : 출력 영상

<br/>

**주요 Threshold Types 열거형 상수**

![image](https://user-images.githubusercontent.com/44194558/140494866-d674101d-5b58-4436-8e95-14d6f4f6966b.png)


In [1]:
import sys
import numpy as np
import cv2

src = cv2.imread('C:/Users/ky_moon/Desktop/vision/ch07/cells.png', cv2.IMREAD_GRAYSCALE)

if src is None:
    print('Image load failed!')
    sys.exit()

_, dst1 = cv2.threshold(src, 100, 255, cv2.THRESH_BINARY)  # retval, dst
_, dst2 = cv2.threshold(src, 210, 255, cv2.THRESH_BINARY)

cv2.imshow('src', src)
cv2.imshow('dst1', dst1)
cv2.imshow('dst2', dst2)
cv2.waitKey()
cv2.destroyAllWindows()

In [4]:
import sys
import numpy as np
import cv2

src = cv2.imread('C:/Users/ky_moon/Desktop/vision/ch07/cells.png', cv2.IMREAD_GRAYSCALE)

if src is None:
    print('Image load failed!')
    sys.exit()

# 트랙바 사용
def on_threshold(pos):
    _, dst = cv2.threshold(src, pos, 255, cv2.THRESH_BINARY)
    cv2.imshow('dst', dst)

cv2.imshow('src', src)
cv2.namedWindow('dst')
cv2.createTrackbar('Threshold', 'dst', 0, 255, on_threshold)
cv2.setTrackbarPos('Threshold', 'dst', 128)

cv2.waitKey()
cv2.destroyAllWindows()

### 자동 이진화 : Otsu 방법

임계값 자동 결정 방법. 입력 영상의 픽셀 값 분포가 두 개의 부류로 구분되는 경우에 최적의 입계값을 결정한다.

입력 영상이 배경과 객체 두 개로 구성되어 있다고 가정 -> Bimodal histogram

**임의의 임계값 T에 의해 나눠지는 두 픽셀 분포 그룹의 분산이 최소가 되는 T를 선택**하는 최적화 알고리즘.

![image](https://user-images.githubusercontent.com/44194558/140496052-4cc69b42-d81e-4019-95d8-b004f5c4f0e0.png)

<br/>

![image](https://user-images.githubusercontent.com/44194558/140496145-80a58174-7dee-456c-8c7e-7518a0836489.png)

<br/>

**Within class variance 최소화 <-> Between class variance 최대화**



![image](https://user-images.githubusercontent.com/44194558/140496358-2e07e5ca-968f-4a8a-ade9-6308f83d69be.png)


모든 T 값에 대해 분산을 계산하여 최적의 T를 구하는 것은 비효율적이기 때문에 recursion을 이용하여 계산의 효율성을 높일 수 있다.




In [ ]:
import sys
import numpy as np
import cv2

src = cv2.imread('C:/Users/ky_moon/Desktop/vision/ch07/rice.png', cv2.IMREAD_GRAYSCALE)

if src is None:
    print('Image load failed!')
    sys.exit()

# Otsu 알고리즘에 의해 결정된 임계값이 th에 저장
# thresh=0으로 설정 (2번 째 인자로 설정한 임계값은 사용되지 않음)
th, dst = cv2.threshold(src, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)  
print("otsu's threshold:", th)  # 131

cv2.imshow('src', src)
cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()

### 적응형 이진화 & 지역 이진화

위의 threshold() 함수는 지정한 임계값을 영상 전체 픽셀에 동일하게 적용하여 이진화를 수행한다. (전역 이진화)

**영상의 특성에 따라서 전역 이진화를 적용하기 어려운 경우가 존재**한다. ex) 균일하지 않은 조명 환경

이와 같은 경우에 전역 이진화를 적용하면 객체와 배경이 적절하게 분리되지 않을 수 있다.

<br/>


![image](https://user-images.githubusercontent.com/44194558/140497388-f73e7583-bbe0-469f-956a-4bdfff2e8593.png)


* 왼편 아래쪽 영역이 다소 어둡게 촬영된 상황.


* ths=100(2번째)인 경우 좌측 하단은 전반적으로 검은색으로 이진화됨


* ths=50(마지막)인 경우 좌측 하단 숫자까지 제대로 이진화가 적용되었지만, 오른쪽 상단의 숫자가 매우 흐려지고 사각형 직선이 심하게 끊어지는 현상이 발생함

불균일한 조명 성분을 가지고 있는 영상에서는 하나의 임계값으로 객체와 배경을 제대로 구분하기 어렵다. **각 픽셀마다 서로 다른 임계값을 사용하는 적응형 이진화 기법을 사용하는 것이 대안**이 될 수 있다.

적응형 이진화는 영상의 모든 픽셀에서 정해진 크기의 사각형 블록 영역을 설정하고, 블록 영역 내부의 픽셀 값 분포로부터 고유한 임계값을 결정하여 이진화하는 방식이다. (x, y) 좌표에서의 임계값 T(x, y)는 다음과 같이 계산된다.


![image](https://user-images.githubusercontent.com/44194558/140499711-84f18438-354d-4d4e-bae8-2a4d199741f5.png)

 * mu(x, y) : (x, y) 주변 블록 영역의 픽셀 값 평균. 가중 평균이나 일반적인 산술 평균 사용.
 
 
 * C : 영상의 특성에 따라 사용자가 설정



**적응형 이진화**

cv2.adaptiveThreshold(src, maxValue, adaptiveMethod, thresholdType, blockSize, C, dst=None) -> dst

`src` : 입력 영상 

`maxValue` : 임계값 함수 최댓값. 보통 255

`adaptiveMethod` : 블록 평균 계산 방법을 지정

`thresholdType` : cv2.TRESHY_BINARY / cv2.THRESH_BINARY_INV 지정

`blockSize` : 블록 크기. 3 이상의 홀수

`C` : 블록 내 평균값 또는 블록 내 가중 평균값에서 뺄 값. (x, y) 픽셀의 임계값으로 T(x, y) = mu_B(x, y) - C 사용

In [7]:
import sys
import numpy as np
import cv2


src = cv2.imread('C:/Users/ky_moon/Desktop/vision/ch07/sudoku.jpg', cv2.IMREAD_GRAYSCALE)

if src is None:
    print('Image load failed!')
    sys.exit()


def on_trackbar(pos):
    bsize = pos
    if bsize % 2 == 0:
        bsize = bsize - 1
    if bsize < 3:
        bsize = 3

    dst = cv2.adaptiveThreshold(src, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                cv2.THRESH_BINARY, bsize, 5)

    cv2.imshow('dst', dst)


cv2.imshow('src', src)
cv2.namedWindow('dst')
cv2.createTrackbar('Block Size', 'dst', 0, 200, on_trackbar)
cv2.setTrackbarPos('Block Size', 'dst', 11)

cv2.waitKey()
cv2.destroyAllWindows()

균일하지 않은 조명의 영향을 해결하기 위해서는 **픽셀 주변에 작은 윈도우를 설정하여 지역 이진화를 수행**한다. 이 때 다음과 같은 사항들을 고려한다.

1. 윈도우의 크기는?


2. 윈도우의 형태는? ex) uniform/gaussian


3. 윈도우를 겹칠 것인가?


4. 윈도우 안에 배경 또는 객체만 존재한다면?



![image](https://user-images.githubusercontent.com/44194558/140498161-a4ac2185-a1ab-41bc-97b0-9f13a179c8c0.png)


In [5]:
import sys
import numpy as np
import cv2

# 입력 영상 불러오기
src = cv2.imread('C:/Users/ky_moon/Desktop/vision/ch07/rice.png', cv2.IMREAD_GRAYSCALE)

if src is None:
    print('Image load failed!')
    sys.exit()

# dst1 : 전역 이진화 (Otsu's method)
_, dst1 = cv2.threshold(src, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)


# dst2 : 지역 이진화 (Otsu's method)
dst2 = np.zeros(src.shape, np.uint8)

# window size (총 16개 window)
bw = src.shape[1] // 4
bh = src.shape[0] // 4

for y in range(4):
    for x in range(4):
        src_ = src[y*bh:(y+1)*bh, x*bw:(x+1)*bw]
        dst_ = dst2[y*bh:(y+1)*bh, x*bw:(x+1)*bw]
        cv2.threshold(src_, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU, dst_)

# 결과 출력
cv2.imshow('src', src)
cv2.imshow('dst1', dst1)
cv2.imshow('dst2', dst2)
cv2.waitKey()
cv2.destroyAllWindows()

## 모폴로지

영상 처리 분야에서 모폴로지는 영상 내부 객체의 형태와 구조를 분석하고 처리하는 기법으로, 수학적 모폴로지라고도 한다. **영상을 형태학적 측면에서 다루는 기법**.

주로 이진화된 영상에서 객체의 모양을 단순화하거나 잡음을 제거하는 등 모양을 변형하는 용도로 사용된다.

**구조 요소**

**모폴로지 연산의 결과를 결정하는 커널**(마스크, 윈도우)로 필터링에서 사용되는 마스크처럼 모폴로지 연산의 동작을 결정하는 행렬. 필요에 따라 원하는 구조 요소를 선택하여 사용할 수 있지만 주로 3 x 3 정방형 구조 요소를 사용한다.


![image](https://user-images.githubusercontent.com/44194558/140502594-a292f1d3-ac24-4281-b29f-220d4e8e7490.png)


 * 고정점 (anchor point)는 모폴로지 연산 결과가 저장될 위치를 나타냄


### 침식 & 팽창

**침식** 연산은 객체 영역의 외관을 골고루 깎아 내는 연산으로 전체적으로 객체 영역은 축소되고 배경은 확대된다. 구조 요소를 영상 전체에 대해 스캔하면서, **구조 요소가 객체 영역 내부에 완전히 포함될 경우 고정점 위치 픽셀을 255로 설정**한다.

객체 영역을 축소시킴으로써 작은 크기의 객체인 잡음 제거 효과가 있다.


![image](https://user-images.githubusercontent.com/44194558/140503881-288ec2d0-2ebc-4b60-9507-18f6d092642b.png)

![image](https://user-images.githubusercontent.com/44194558/140504046-6ffa96b6-b435-48b3-845d-d55ba2785497.png)


<br/>
<br/>


**팽창** 연산은 객체의 외곽을 확대하는 연산으로 객체 영역은 확대되고, 배경 영역은 축소된다. 구조 요소를 영상 전체에 대해 이동시키면서, **구조 요소와 객체 영역이 한 픽셀이라도 만날 경우 고정점 위치 픽셀을 255로 설정**한다.

객체 영역을 확대시킴으로써 객체 내부의 구멍이 채워지는 효과가 있다.


![image](https://user-images.githubusercontent.com/44194558/140503926-ea5abd02-6e50-4e6c-a8e7-1a806779dcf4.png)

![image](https://user-images.githubusercontent.com/44194558/140504127-3bd4adea-b215-41e1-b649-64df1cd81590.png)



**모폴로지 침식 연산**

cv2.erode(src, kernel, dst=None, anchor=None, iterations=None, borderType=None, borderValue=None) -> dst

`kernel` : 구조 요소. None을 지정하면 3x3 사각형 구조 요소 사용. getStructuringElement() 함수에 의해 생성 가능.

`anchor` : 고정점 위치. default=(-1, -1)을 사용하면 중앙점을 사용.

`iterations` : 반복 횟수. default=1

`borderType` : 가장자리 픽셀 확장 방식. default=cv2.BORDER_CONSTANT

`borderValue` : cv2.BORDER_CONSTANT인 경우, 확장된 가장자리 픽셀을 채울 값.

**모폴로지 팽창 연산**

cv2.dilate(src, kernel, dst=None, anchor=None, iterations=None, borderType=None, borderValue=None) -> dst

`kernel` : 구조 요소. None을 지정하면 3x3 사각형 구조 요소 사용. getStructuringElement() 함수에 의해 생성 가능.

`anchor` : 고정점 위치. default=(-1, -1)을 사용하면 중앙점을 사용.

`iterations` : 반복 횟수. default=1

`borderType` : 가장자리 픽셀 확장 방식. default=cv2.BORDER_CONSTANT

`borderValue` : cv2.BORDER_CONSTANT인 경우, 확장된 가장자리 픽셀을 채울 값.

**모폴로지 구조 요소 생성**

cv2.getStructuringElement(shape, ksize, anchor=None) -> retval

`shape` : 구조 요소 모양을 나타내는 플래그.

`ksize` : 구조 요소 크기. (w, h) 튜플.

`ahchor` : 고정점 좌표. (-1, -1)을 지정하면 중앙점 사용.

`retval` : 0, 1로 구성된 cv2.CV_8UC1 타입의 numpy.ndarray 행렬. (1의 위치가 구조 요소의 모양을 결정)

In [8]:
import sys
import numpy as np
import cv2


src = cv2.imread('C:/Users/ky_moon/Desktop/vision/ch07/circuit.bmp', cv2.IMREAD_GRAYSCALE)

if src is None:
    print('Image load failed!')
    sys.exit()

se = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 3))
dst1 = cv2.erode(src, se)  # 침식

dst2 = cv2.dilate(src, None)  # 팽창

cv2.imshow('src', src)
cv2.imshow('dst1', dst1)
cv2.imshow('dst2', dst2)
cv2.waitKey()
cv2.destroyAllWindows()

### 열기 & 닫기

**열기 연산**은 입력 영상에 대해 침식 연산을 수행한 후, 다시 팽창 연산을 수행하는 연산.

   * 입력 이진 영상에 존재하는 작은 크기의 객체가 효과적으로 제거됨. (한 두 픽셀짜리 영역이 제거된 후, 팽창 연산이 수행)

**닫기 연산**은 입력 영상에 대해 팽창 연산을 수행한 후, 다시 침식 연산을 수행하는 연산.

   * 객체 내부의 작은 구멍을 제거함. (객체 내부의 작은 구멍이 메워진 후, 침식 연산이 수행)

각각 침식과 팽창 연산이 한 번씩 적용되기 때문에 객체 영역의 크기가 크게 바뀌지 않으나, 침식과 팽창이 적용되는 순서에 따라 서로 다른 효과가 발생한다.


![image](https://user-images.githubusercontent.com/44194558/140506775-f3f8f62d-2e98-4437-94a5-8388107c74c2.png)


**범용 모폴로지 연산 함수**

cv2.morphologyEx(src, op, kernel, dst=None, anchor=None, iterations=None, borderType=None, borderValue=None) -> dst

`op` : 모폴로지 연산 플래그

`kernel` : 커널


![image](https://user-images.githubusercontent.com/44194558/140506976-56399d73-931b-4321-a9b8-8f3f2d004d80.png)

In [11]:
import sys
import numpy as np
import cv2


src = cv2.imread('C:/Users/ky_moon/Desktop/vision/ch07/rice.png', cv2.IMREAD_GRAYSCALE)

if src is None:
    print('Image load failed!')
    sys.exit()

# src 영상에 지역 이진화 수행 
dst1 = np.zeros(src.shape, np.uint8)

bw = src.shape[1] // 4
bh = src.shape[0] // 4

for y in range(4):
    for x in range(4):
        src_ = src[y*bh:(y+1)*bh, x*bw:(x+1)*bw]
        dst_ = dst1[y*bh:(y+1)*bh, x*bw:(x+1)*bw]
        cv2.threshold(src_, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU, dst_)

cnt1, _ = cv2.connectedComponents(dst1)
print('cnt1:', cnt1)

dst2 = cv2.morphologyEx(dst1, cv2.MORPH_OPEN, None)
#dst2 = cv2.erode(dst1, None)
#dst2 = cv2.dilate(dst2, None)

cnt2, _ = cv2.connectedComponents(dst2)
print('cnt2:', cnt2)

cv2.imshow('src', src)
cv2.imshowㅡ('dst1', dst1)
cv2.imshow('dst2', dst2)
cv2.waitKey()
cv2.destroyAllWindows()


cnt1: 113
cnt2: 99


## 레이블링

영상 내부에 있는 각 객체의 위치, 크기, 모양 등 특징을 분석할 때 사용되는 **영역 기반 모양 분석**으로 객체 인식의 전처리 과정으로 사용한다. 레이블맵, 바운딩 박스, 픽셀 개수, 무게 중심 좌표를 반환한다.

영상의 이진화를 통해 주요 객체와 배경 영역을 구분한 이후, 각각의 객체를 구분하고 분석하는 작업이 필요하다. 이때 레이블링을 통해 영상 내에 존재하는 **객체 픽셀 집합에 고유 번호를 매기는 작업**을 수행한다. **연결된 구성 요소 레이블링(conncected components labeling)**이라고도 함. 

입력 영상의 픽셀 값이 0이면 배경, 0이 아니면 객체 픽셀로 인식한다. 하나의 객체는 한 개 이상의 인접한 픽셀로 이루어지고, 하나의 객체를 구성하는 모든 픽셀에는 같은 레이블 번호가 지정된다. 즉, 서로 연결되어 있는 객체 픽셀에 고유한 번호를 지정하는 과정이다.

**픽셀의 연결 관계**는 크게 두 가지 방식으로 정의된다.

1. 4-이웃 연결 관계 : 특정 픽셀의 상하좌우로 붙어 있는 픽셀끼리 연결되어 있다고 정의


2. 8-이웃 연결 관계 : 상하좌우 + 대각선 방향으로 인접한 픽셀도 연결되어 있다고 정의

![image](https://user-images.githubusercontent.com/44194558/140599510-4b3f3344-e753-4188-80c7-827515b07b86.png)

<br/>


**레이블링 알고리즘의 입력&출력**

<br/>

![image](https://user-images.githubusercontent.com/44194558/140599526-72292073-f8e2-4335-8318-3462efd2c3d4.png)



**레이블링 함수**

cv2.connectedComponents(image, labels=None, connectivity=None, ltype=None) -> retval, labels

`image` : 8비트 1채널 영상 (이진 영상)

`labels` : 레이블 맵 행렬. 입력 영상과 같은 크기의 numpy.ndarray

`connectivity` : 4 또는 8. default=8

`ltype` : labels 타입. default=cv2.CV_32S

`retval` : 객체 개수. N을 반환하면 [0, N-1]의 레이블이 존재. 0은 배경을 의미하기 때문에 실제 흰색의 객체 개수는 N-1개

**객체 정보를 반환하는 레이블링 함수**

각각의 객체 영역이 어느 위치에 어느 정도의 크기로 존재하는 지 확인

cv2.connectedComponentsWithStats(image, labels=None, stats=None, centroids=None, connectivity=None, ltype=None) -> retval, labels, stats, centroids

`image` : 8비트 1채널 영상 (이진 영상)

`labels` : 레이블 맵 행렬. 입력 영상과 같은 크기의 numpy.ndarray

`stats` : 각 객체의 바운딩 박스. 픽셀 개수 정보를 담은 행렬. (N, 5) 크기의 numpy.ndarray, dtype=numpy.int32

`centroids` : 각 객체의 무게 중심 위치 정보를 담은 행렬. (N, 2) 크기의 numpy.ndarray, dtype=numpy.float64

`connectivity` : 4 또는 8. default=8

`ltype` : labels 타입. default=cv2.CV_32S

`retval` : 객체 개수. N을 반환하면 [0, N-1]의 레이블이 존재. 0은 배경을 의미하기 때문에 실제 흰색의 객체 개수는 N-1개

<br/>

![image](https://user-images.githubusercontent.com/44194558/140599833-401a8526-de85-4957-a0cc-d7c7ec7b5e4c.png)

 * stats 행렬 2번째 행 [0, 0, 4, 3, 10]  : 1번 객체를 감싸는 바운딩 박스가 (0, 0) 좌표에서 시작하여 가로 크기가 4, 세로 크기가 3인 사각형이고 1번 객체의 픽셀 개수는 10
 
 * 무게 중심 좌표는 객체 픽셀의 x, y 좌표를 모두 더한 후 픽셀 개수로 나눈 값

In [12]:
import sys
import numpy as np
import cv2


src = cv2.imread('C:/Users/ky_moon/Desktop/vision/ch07/keyboard.bmp', cv2.IMREAD_GRAYSCALE)

if src is None:
    print('Image load failed!')
    sys.exit()

_, src_bin = cv2.threshold(src, 0, 255, cv2.THRESH_OTSU)  # 영상 이진화

cnt, labels, stats, centroids = cv2.connectedComponentsWithStats(src_bin)

dst = cv2.cvtColor(src, cv2.COLOR_GRAY2BGR)

for i in range(1, cnt):
    (x, y, w, h, area) = stats[i]

    if area < 20:
        continue

    cv2.rectangle(dst, (x, y, w, h), (0, 255, 255))

cv2.imshow('src', src)
cv2.imshow('src_bin', src_bin)
cv2.imshow('dst', dst)
cv2.waitKey()ㅡ
cv2.destroyAllWindows()

## 외곽선 검출

객체의 외곽선 좌표를 모두 추출하는 작업. Boundary tracking. Contour tracing. OpenCV의 외곽선 검출 기능은 객체의 외곽선 픽셀 좌표를 모두 추출하여 계층 정보와 함께 반환한다. 

외곽선은 **객체 영역 픽셀 중에서 배경 영역과 인접한 일련의 픽셀**을 의미한다. 보통 검은색 배경 안에 있는 흰색 객체 영역에서 가장 최외곽에 있는 픽셀을 외곽선으로 정의한다. 흰색 객체 영역 안에 검은색 배경 영역인 홀이 존재하면 홀을 둘러싸는 객체 픽셀들도 외곽선으로 정의할 수 있다. 즉, 객체의 외곽선은 **객체 바깥쪽 외곽선과 안쪽 홀 외곽선으로 구분**할 수 있다. (외곽선의 계층 구조를 표현할 수 있음)

개체 하나의 외곽선은 numpy.int32 타입의 (K, 1, 2) 크기를 갖는 numpy.ndarray (K=외곽선 좌표 개수). 여러 개의 외곽선은 객체 하나의 외곽선을 원소로 갖는 리스트로 len(리스트) = 전체 외곽선의 개수.

**외곽선 검출 함수**

cv2.findContours(image, mode, method, contours=None, hierarchy=None, offset=None) -> contours, hierarchy

`image` : 입력 영상. non-zero 픽셀을 객체로 간주함.

`mode` : 외곽선 검출 모드

`method` : 외곽선 근사화 방법

`contours` : 검출된 외곽선 좌표. numpy.ndarray로 구성된 리스트. len(contours)=전체 외곽선의 개수 N

`hierarchy` : 외곽선 계층 정보. (1, N, 4) 크기의 numpy.ndarray. hierarchy[0, i, 0] ~ hierarchy[0, i, 3]이 순서대로 i 번째 외곽선의 next, prev, child, parent 외곽선 인덱스를 가리킴.

`offset` : 좌표값 이동 offset

![image](https://user-images.githubusercontent.com/44194558/140600442-25ffaca3-bf66-499c-9a07-bd9ae94617b2.png)


![image](https://user-images.githubusercontent.com/44194558/140600457-4d1e58b0-7a5c-48a7-bfed-cba087ec7b00.png)

![image](https://user-images.githubusercontent.com/44194558/140600472-00fbfd0b-13ac-49cc-a65e-d5f7f03264f0.png)

![image](https://user-images.githubusercontent.com/44194558/140600502-e1ecfc6d-f15d-4c20-9f4a-db041be7b361.png)


![image](https://user-images.githubusercontent.com/44194558/140600513-ffb42d65-97e4-426e-a944-f1180b471827.png)

**외곽선 그리기**

cv2.drawContours(image, contours, contourIdx, color, thickness=None, lineType=None, hierarchy=None, maxLevel=None, offset=None) -> image

`contours` : cv2.findContours()로 계산한 외곽선 좌표 정보

`contourIdx` : 외곽선 인덱스. -1 지정시 모든 외곽선을 그림.

`color` : 외곽선 색상

`thickness` : 외곽선 두께. 음수로 설정하면 내부를 채운다.

`lineType` : LINE_4, LINE_8, LINE_AA

`hierarchy` : 외곽선 계층 정보

`maxLevel` : 그리기를 수행할 최대 외곽선 레벨. 0이면 contourIdx로 지정된 외곽선만 그린다.

In [28]:
import sys
import random
import numpy as np
import cv2


src = cv2.imread('C:/Users/ky_moon/Desktop/vision/ch07/contours.bmp', cv2.IMREAD_GRAYSCALE)

if src is None:
    print('Image load failed!')
    sys.exit()

contours, hier = cv2.findContours(src, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)  # 계층 정보 o

dst = cv2.cvtColor(src, cv2.COLOR_GRAY2BGR)

idx = 0
while idx >= 0:
    c = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
    cv2.drawContours(dst, contours, idx, c, 2, cv2.LINE_8, hier)
    idx = hier[0, idx, 0]

cv2.imshow('src', src)
cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()


![image](https://user-images.githubusercontent.com/44194558/140601467-4796d297-ff39-4b2d-bbb0-ff8690e0295b.png)

바깥쪽 외곽선만 검출

In [31]:
import sys
import random
import numpy as np
import cv2


src = cv2.imread('C:/Users/ky_moon/Desktop/vision/ch07/contours.bmp', cv2.IMREAD_GRAYSCALE)

if src is None:
    print('Image load failed!')
    sys.exit()

contours, hier = cv2.findContours(src, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)  # 계층 정보 x

dst = cv2.cvtColor(src, cv2.COLOR_GRAY2BGR)

idx = 0
while idx >= 0:
    c = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
    cv2.drawContours(dst, contours, idx, c, 2, cv2.LINE_8, hier)
    idx = hier[0, idx, 0]

cv2.imshow('src', src)
cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()


![image](https://user-images.githubusercontent.com/44194558/140601517-1d19ee2a-336c-47b0-ac68-4b3722f70a0a.png)

안 쪽 영역까지 잘 검출하나 계층 정보를 반영하지 않음

In [32]:
import sys
import random
import numpy as np
import cv2


src = cv2.imread('C:/Users/ky_moon/Desktop/vision/ch07/contours.bmp', cv2.IMREAD_GRAYSCALE)

if src is None:
    print('Image load failed!')
    sys.exit()

contours, hier = cv2.findContours(src, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)  # 계층 정보 x

dst = cv2.cvtColor(src, cv2.COLOR_GRAY2BGR)

idx = 0
while idx >= 0:
    c = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
    cv2.drawContours(dst, contours, idx, c, 2, cv2.LINE_8, hier)
    idx = hier[0, idx, 0]

cv2.imshow('src', src)
cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()


![image](https://user-images.githubusercontent.com/44194558/140601430-5ad7ba3c-5ca5-41f5-bd0b-c2c72c69f33f.png)

안 쪽 영역도 잘 검출. 부모/자식 관계를 잘 표현하고 있음 (같은 색으로 표현함)

In [20]:
import sys
import random
import numpy as np
import cv2


src = cv2.imread('C:/Users/ky_moon/Desktop/vision/ch07/contours.bmp', cv2.IMREAD_GRAYSCALE)

if src is None:
    print('Image load failed!')
    sys.exit()

contours, hier = cv2.findContours(src, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_NONE)  # 계층 정보 o

dst = cv2.cvtColor(src, cv2.COLOR_GRAY2BGR)

idx = 0
while idx >= 0:
    c = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
    cv2.drawContours(dst, contours, idx, c, 2, cv2.LINE_8, hier)
    idx = hier[0, idx, 0]

cv2.imshow('src', src)
cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()


![image](https://user-images.githubusercontent.com/44194558/140601340-e080cbbd-4e30-40ba-97f9-0372fdb8619a.png)



In [33]:
import sys
import random
import numpy as np
import cv2


src = cv2.imread('C:/Users/ky_moon/Desktop/vision/ch07/milkdrop.bmp', cv2.IMREAD_GRAYSCALE)

if src is None:
    print('Image load failed!')
    sys.exit()

_, src_bin = cv2.threshold(src, 0, 255, cv2.THRESH_OTSU)

contours, _ = cv2.findContours(src_bin, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)  # 계층 정보 x

h, w = src.shape[:2]
dst = np.zeros((h, w, 3), np.uint8)

for i in range(len(contours)):
    c = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
    cv2.drawContours(dst, contours, i, c, 1, cv2.LINE_AA)

cv2.imshow('src', src)
cv2.imshow('src_bin', src_bin)
cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()


![image](https://user-images.githubusercontent.com/44194558/140601577-8fcb1f0e-5b79-44b1-b0ee-904e697a2f49.png)


### 외곽선 처리 관련 함수

외곽선 좌표 정보를 이용하는 여러 함수들. 함수로부터 얻은 정보를 이용하여 객체의 모양을 판단할 수 있다.

![image](https://user-images.githubusercontent.com/44194558/140601611-852a6b31-d377-46a8-b30c-bfb65bc1452f.png)

**외곽선 길이 구하기**

cv2.arcLength(curve, closed) -> retval

`curve` : 외곽선 좌표. (K, 1, 2) 크기의 외곽선 좌표

`closed` : Ture이면 폐곡선으로 간주

`retval` : 외곽선 길이

**면적 구하기**

cv2.contourArea(contour, oriented=None) -> retval

`contour` : (K, 1, 2) 크기의 외곽선 좌표

`oriented` : True이면 외곽선 진행 방향에 따라 부호 있는 면적을 반환. default=False

`retval` : 외곽선으로 구성된 영역의 면적

**바운딩 박스 구하기**

cv2.boundingRect(array) -> retval

`array` : 외곽선 좌표

`retval` : (x, y, w, h) 튜플의 사각형 정보

**바운딩 서클 구하기**

cv2.minEnclosingCircle(points) -> center, radius

`points` : 외곽선 좌표

`center` : 바운딩 서클의 중심 좌표. (x, y) 튜플

`radius` : 바운딩 서클의 반지름. 실수형

![image](https://user-images.githubusercontent.com/44194558/140601995-7f547cda-2962-4985-972a-ac8005419533.png)

**외곽선 근사화**

cv2.approxPolyDP(curve, epsilon, closed, approxCurve=None) -> approxCurve

`curve` : 입력 곡선 좌표. shape=(K, 1, 2)

`epsilon` : 근사화 정밀도 조절. 입력 곡선과 근사화 곡선의 최대 거리.

`closed` : True이면 폐곡선으로 인식

`approx` : 근사화된 곡선 좌표. shape=(K', 1, 2)

![image](https://user-images.githubusercontent.com/44194558/140602014-e0f8dd5b-6901-4087-88d1-c6f54503c5f2.png)

![330px-Douglas-Peucker_animated](https://user-images.githubusercontent.com/44194558/140601933-b8356d27-c1cf-496d-bf22-113e59b560d6.gif)


**Convex 검사**

cv2.isContourConvex(contour) -> retval

`contour` : 입력 곡선 좌표

`retval` : 컨벡스이면 True

### 다각형 검출 프로그램

**구현 순서**



1. 이진화


2. 외곽선 좌표 찾기


3. 외곽선 근사화


4. 잡음 객체, 컨벡스가 아닌 개체 제외



5. 꼭지점의 개수를 확인

  - 삼각형, 사각형 검출
  
  - 원 판별

In [ ]:
import math
import cv2


def setLabel(img, pts, label):
    (x, y, w, h) = cv2.boundingRect(pts)  # 바운딩 박스 정보
    pt1 = (x, y)
    pt2 = (x + w, y + h)
    cv2.rectangle(img, pt1, pt2, (0, 0, 255), 1)
    cv2.putText(img, label, pt1, cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 255))


def main():
    img = cv2.imread('C:/Users/ky_moon/Desktop/vision/ch07/polygon.bmp', cv2.IMREAD_COLOR)

    if img is None:
        print('Image load failed!')
        return

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, img_bin = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)  # 영상 이진화
    contours, _ = cv2.findContours(img_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)  # 외곽선 검출

    for pts in contours:
        if cv2.contourArea(pts) < 400:  #  검출된 객체의 면적이 너무 작으면 무시
            continue

        approx = cv2.approxPolyDP(pts, cv2.arcLength(pts, True)*0.02, True)  # 외곽선 근사화

        vtc = len(approx)  # 꼭짓점의 개수

        if vtc == 3:
            setLabel(img, pts, 'TRI')
            
        elif vtc == 4:
            setLabel(img, pts, 'RECT')
        
        # 원 판별 방법
        else:
            length = cv2.arcLength(pts, True)
            area = cv2.contourArea(pts)
            ratio = 4. * math.pi * area / (length * length)  # 도형의 넓이와 외곽선 길이의 비율을 검사

            if ratio > 0.85:  # 정해진 외곽선 길이에 대한 넓이 비율이 가장 큰 형태가 원
                setLabel(img, pts, 'CIR')

    cv2.imshow('img', img)
    cv2.waitKey()
    cv2.destroyAllWindows()


if __name__ == '__main__':
    main()

**원 판별 방법**


![image](https://user-images.githubusercontent.com/44194558/140602408-fe8ef197-31f7-4e45-a639-bb8a843eee26.png)

   * A : 도형의 넓이
   

   * P : 외곽선 길이
   
   
제일 오른 쪽 값이 1에 가까울 수록 원으로 판단

### 명함 인식 프로그램

영상에서 명함을 검출하고, 명함 안의 텍스트를 인식하는 프로그램

명함은 흰색이고, 충분히 크게 촬영되었으며, 각진 사각형 모양이라고 가정함

**구현 순서**

1. 명함 검출


2. 이진화


3. 외곽선 검출 & 다각형 근사화


4. 투시 변환

**명함 검출**


1. 입력 영상에 대해 Otsu 자동 이진화 수행


2. 명함의 외곽선 검출

   2.1 근사화
   
   2.2 사각형 검출
   
   
3. 명함의 네 모서리 점을 직사각형의 네 모서리로 매핑

**명함 텍스트 인식**


광학 문자 인식(OCR) 라이브러리인 Tesseract 이용


Tesseract-ocr 설치하기

1. tesseract-ocr-w64-setup-v5.0.0-alpha.20200328 파일 다운로드 
   (https://digi.bib.uni-mannheim.de/tesseract/tesseract-ocr-w64-setup-v5.0.0-alpha.20200328.exe)
   
   
2. 설치 시 "Additional script data" 항목에서 "Hangul Script", "Hangul vertical script" 항목 체크,
   "Additional language data" 항목에서 "Korean" 항목 체크.
   
   
3. 설치 후 시스템 환경변수 PATH에 Tesseract 설치 폴더 추가
   (e.g.) c:\Program Files\Tesseract-OCR
   
   
4. 설치 후 시스템 환경변수에 TESSDATA_PREFIX를 추가하고, 변수 값을 <Tesseract-DIR>\tessdata 로 설정
    
    
5. <Tesseract-DIR>\tessdata\script\ 폴더에 있는 Hangul.traineddata, Hangul_vert.traineddata 파일을
   <Tesseract-DIR>\tessdata\ 폴더로 복사
       
       
6. 명령 프롬프트 창에서 pip install pytesseract 명령 입력

![image](https://user-images.githubusercontent.com/44194558/140603088-d3af98e2-bcac-4a52-a049-6216c5991e59.png)

In [ ]:
import sys
import random
import numpy as np
import cv2
import pytesseract


def reorderPts(pts):
    idx = np.lexsort((pts[:, 1], pts[:, 0]))  # 칼럼0 -> 칼럼1 순으로 정렬한 인덱스를 반환
    pts = pts[idx]  # x좌표로 정렬

    if pts[0, 1] > pts[1, 1]:  # pts[0]의 y 좌표보다 pts[1]의 y좌표가 더 커야함
        pts[[0, 1]] = pts[[1, 0]]  # 좌표 위치 바꾸기

    if pts[2, 1] < pts[3, 1]:  # pts[2]의 y 좌표보다 pts[3]의 y좌표가 더 커야함
        pts[[2, 3]] = pts[[3, 2]]

    return pts


# 영상 불러오기
filename = 'C:/Users/ky_moon/Desktop/vision/ch07/namecard1.jpg'
if len(sys.argv) > 1:
    filename = sys.argv[1]

src = cv2.imread(filename)

if src is None:
    print('Image load failed!')
    sys.exit()

# 출력 영상 설정
dw, dh = 720, 400
srcQuad = np.array([[0, 0], [0, 0], [0, 0], [0, 0]], np.float32)
dstQuad = np.array([[0, 0], [0, dh], [dw, dh], [dw, 0]], np.float32)  # 좌측 상단부터 반 시계 방향으로 pts[0], pts[1], pts[2], pts[3]
dst = np.zeros((dh, dw), np.uint8)

# 입력 영상 전처리
src_gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
_, src_bin = cv2.threshold(src_gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

# 외곽선 검출 및 명함 검출
contours, _ = cv2.findContours(src_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

cpy = src.copy()
for pts in contours:
    # 너무 작은 객체는 무시
    if cv2.contourArea(pts) < 1000:
        continue

    # 외곽선 근사화
    approx = cv2.approxPolyDP(pts, cv2.arcLength(pts, True)*0.02, True)

    # 컨벡스가 아니고, 사각형이 아니면 무시
    if not cv2.isContourConvex(approx) or len(approx) != 4:
        continue

    cv2.polylines(cpy, [approx], True, (0, 255, 0), 2, cv2.LINE_AA)
    srcQuad = reorderPts(approx.reshape(4, 2).astype(np.float32))

pers = cv2.getPerspectiveTransform(srcQuad, dstQuad)
dst = cv2.warpPerspective(src, pers, (dw, dh))

dst_gray = cv2.cvtColor(dst, cv2.COLOR_BGR2GRAY)
print(pytesseract.image_to_string(dst_gray, lang='Hangul+eng'))

cv2.imshow('src', src)
cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()